In [1]:
#Add the following into requirements.txt
import requests
import boto3
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

In [2]:
ddb = boto3.client('dynamodb')
# TODO: You should pickup the subreddit from the event param from the handler
# For now hardcoding it. 

subreddit = 'funny'
get_item = { 
            "subreddit": {"S": "funny" } 
       }

headers = {'User-Agent': 'funnyAPI/0.0.1'}
#Initializing without the actual username and password 
data = {
    'grant_type' : 'password',
    'username': "",
    'password': ""
}

# TODO: These should be picked from environment. Hardcoding it for now:
reddit_auth_url = 'https://www.reddit.com/api/v1/access_token'
reddit_api_url_hot = 'https://oauth.reddit.com/r/place_holder_value/hot'
reddit_api_url_new = 'https://oauth.reddit.com/r/place_holder_value/new'

In [3]:
def deserialize_db_item(item):
    new_item = {}
    for key in item:        
        new_item[key] = extract_value(item[key])
    
    return new_item

def extract_value(dictionary):    
    data_type, value = list(dictionary.keys())[0], list(dictionary.values())[0]
   
    if data_type == 'S':
        return value     

In [4]:
## The actual handler code may start from here:

try:
    # TODO: Have to pick up the table name from environment. 
    # Hardcoding it for now
    response = ddb.get_item(TableName='RedditAccountsTable-dev', Key=get_item)   
    item = deserialize_db_item(response['Item'])   
    CLIENT_ID = item['personal_use_script']
    SECRET_KEY = item['secret_key']
    username = item['username']
    password = item['password']   
    
except Exception as e:
    print(f"Failed with exception: {e.args[0]}")

In [5]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

In [6]:
data['username'] = username
data['password'] = password

In [7]:
# Request for access token from Reddit API
res = requests.post(reddit_auth_url, auth=auth, data=data, headers=headers)

In [8]:
ACCESS_TOKEN = res.json()['access_token']

In [9]:
headers['Authorization'] = f'bearer {ACCESS_TOKEN}'

In [10]:
reddit_api_url_hot = reddit_api_url_hot.replace('place_holder_value', subreddit)

In [11]:
res = requests.get(reddit_api_url_hot, headers=headers)

In [12]:
for post in res.json()['data']['children']:
    pp.pprint(post['data']['title'])

"ANNOUNCEMENT: We're making some changes to how comics are handled."
('Subreddit Of The Month [August 2021]: /r/yourpointbeing/. Know of a small '
 '(under 20,000 subscribers) humor-based subreddit that deserves a month in '
 'the spotlight? Link it inside!')
"Don't tell me what to do"
("A picture to commemorate my husband's first day of his office reopening. "
 'They grow up so fast. 😢')
'Returning to Normal'
'Gonna have to call that repairman back'
'Oh, so you can just walk up.'
'dick falls out while running'
'Any more suggestions? [OC]'
'Seconds before disaster struck'
'A squirrel won today'
'I trust the dog'
'An ode to Danny DeVito'
'Meth: Not even once'
'Car maths'
'A truly niche bumper sticker'
'This is crazy 😧😧😧'
'Not again.'
'Spruced up the bathroom at work.'
'Someone else posted this on my account, don’t stalk me'
'My co worker got a vasectomy, so we got him a cake to celebrate the occasion'
('My sister in law wanted a wedding cape. So she could pose as a villian '
 'during th

In [13]:
# If post contains images you can extract the image this way: 
# pp.pprint(res.json()['data']['children'][4]['data']['url'])
pp.pprint(res.json()['data']['children'][4]['data']['url'])

'https://i.redd.it/t6ugicupzcg71.jpg'


In [14]:
# If the post contains video, you can extract the url this way.
# I think fallback_url gives best resolution video ?:
pp.pprint(res.json()['data']['children'][5]['data']['media'])
pp.pprint(res.json()['data']['children'][5]['data']['is_video'])

None
False


In [15]:
pp.pprint(res.json()['data']['children'][5]['data'])

{ 'all_awardings': [ { 'award_sub_type': 'GLOBAL',
                       'award_type': 'global',
                       'awardings_required_to_grant_benefits': None,
                       'coin_price': 100,
                       'coin_reward': 0,
                       'count': 1,
                       'days_of_drip_extension': 0,
                       'days_of_premium': 0,
                       'description': "Shows the Silver Award... and that's "
                                      'it.',
                       'end_date': None,
                       'giver_coin_reward': None,
                       'icon_format': None,
                       'icon_height': 512,
                       'icon_url': 'https://www.redditstatic.com/gold/awards/icon/silver_512.png',
                       'icon_width': 512,
                       'id': 'gid_1',
                       'is_enabled': True,
                       'is_new': False,
                       'name': 'Silver',
              

In [19]:
pp.pprint(res.json()['data']['children'][7]['data']['media']['reddit_video']['duration'])

29


In [26]:
df_hot = pd.DataFrame()
total_duration = 0
for post in res.json()['data']['children']:
    if post['data']['is_video']:
        df_hot = df_hot.append({
    #         'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['title'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'url': post['data']['url'],
        }, ignore_index=True)
        
        total_duration += post['data']['media']['reddit_video']['duration'] 
    
df_hot

,title,selftext,upvote_ratio,ups,downs,score,url
0,"Oh, so you can just walk up.","Oh, so you can just walk up.",0.94,44184.0,0.0,44184.0,https://v.redd.it/bs62f4e9dcg71
1,dick falls out while running,dick falls out while running,0.87,28769.0,0.0,28769.0,https://v.redd.it/6m0v1c4wubg71
2,Work It,Work It,0.88,241.0,0.0,241.0,https://v.redd.it/mtgcitvv4eg71
3,Gonna tell my kids this is how India won gold ...,Gonna tell my kids this is how India won gold ...,0.89,160.0,0.0,160.0,https://v.redd.it/b8bbgnxgieg71


In [25]:
pp.pprint(post['data']['media']['reddit_video']['duration'])

24


In [27]:
total_duration

106

In [19]:
print(post['data']['name'], post['kind'], post['data']['id'])

t3_p11zhb t3 p11zhb


# Trying out the /new endpoint. Not required to add to handler

In [20]:
reddit_api_url_new = reddit_api_url_new.replace('place_holder_value', subreddit)
reddit_api_url_new

'https://oauth.reddit.com/r/funny/new'

In [21]:
res = requests.get(reddit_api_url_new, headers=headers, params={'limit':'100'})

In [22]:
res.json()

{'kind': 'Listing',
 'data': {'after': 't3_p0u4ha',
  'dist': 100,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'funny',
     'selftext': '',
     'author_fullname': 't2_13ek4a',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'I have that affect on people',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/funny',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 140,
     'top_awarded_type': None,
     'hide_score': True,
     'name': 't3_p135gn',
     'quarantine': False,
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.67,
     'author_flair_background_color': None,
     'subreddit_type': 'public',
     'ups': 1,
     'total_awards_received': 0,
     'media_embed': {},
     'thumbnail_width': 140,
     'author_flair_template_id': None,
     'is_original

In [23]:
df_new = pd.DataFrame()

for post in res.json()['data']['children']:
    if post['data']['is_video']:
        df_new = df_new.append({
    #         'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['title'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'url': post['data']['url'],
        }, ignore_index=True)
    
df_new

,title,selftext,upvote_ratio,ups,downs,score,url
0,Congrats. Very well deserved.,Congrats. Very well deserved.,1.00,5.0,0.0,5.0,https://v.redd.it/l0wjy9wwjcg71
1,Lightning hit very close to my house. This is ...,Lightning hit very close to my house. This is ...,0.73,10.0,0.0,10.0,https://v.redd.it/i5ev03wygcg71
2,EXCUSE ME! I'm eating here!,EXCUSE ME! I'm eating here!,0.50,0.0,0.0,0.0,https://v.redd.it/ifwenmzxfcg71
3,What are men really thinking?,What are men really thinking?,0.97,145.0,0.0,145.0,https://v.redd.it/mby7qqjxdcg71
4,"Oh, so you can just walk up.","Oh, so you can just walk up.",0.95,139.0,0.0,139.0,https://v.redd.it/bs62f4e9dcg71
5,johnny the kitten,johnny the kitten,0.88,24.0,0.0,24.0,https://v.redd.it/t32m9g60dcg71
6,Goodluck to all students..,Goodluck to all students..,0.92,49.0,0.0,49.0,https://v.redd.it/n2s3b67tacg71
7,Bird be lagging.,Bird be lagging.,0.88,26.0,0.0,26.0,https://v.redd.it/nbx2pcttbcg71
8,Guy from Boston yelling at a demon cat.,Guy from Boston yelling at a demon cat.,0.87,69.0,0.0,69.0,https://v.redd.it/mwlyhz1jbcg71
9,Street Fighter: Cats Edition.,Street Fighter: Cats Edition.,0.90,63.0,0.0,63.0,https://v.redd.it/ss3hesa59cg71


In [24]:
res = requests.get(reddit_api_url_new, headers=headers, params={'limit':'100', 'after': post['data']['name']})

In [25]:
res.json()

{'kind': 'Listing',
 'data': {'after': 't3_p0m816',
  'dist': 100,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'funny',
     'selftext': '',
     'author_fullname': 't2_aeo8oo1z',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'Rock em , Knock em , Sock em Featuring Jimmy smith the worst boxer',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/funny',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 78,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_p0tvdl',
     'quarantine': False,
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.96,
     'author_flair_background_color': None,
     'subreddit_type': 'public',
     'ups': 1583,
     'total_awards_received': 2,
     'media_embed': {},
     'thumbnail_width': 140,
     'author_

In [26]:
for post in res.json()['data']['children']:
    print(post['data']['title'])

Rock em , Knock em , Sock em Featuring Jimmy smith the worst boxer
Rap Roundtable - SNL
Think jail is just a room.
Australia's favourite restaurant.
That one time my dog looked like an Argonian from The Elder Scrolls games
This picture of me and my friends
That’s one way to get a gold medal.
Which Friday Movie?
Watching the Olympics, nephew says…’IS THAT PEE-WEE?’
Yep, definitely just a harmless and cute plunger
He's real!
I’ve been here too long. However my boxers were nice and warm, it was quite lovely.
Man gets baptised in freezing water
In honor of Trevor Moore, here is my favorite WKUK sketch: [Baby Skull Seeking Bullet]
i cant remember where i found this but it’s funny so credit to that guy!
I made a homemade pizza today my first attempt.
The guy in white's interest level increases with each picture.
Fixed it
Ash came home from ER
Fade me bro.
Enovironmentalists are getting a bit more aggressive.
How To Win A Street Fight Against Your Friend - [ Action Comedy Short Film ]
Where b

In [27]:
df_new.sort_values(['score', 'upvote_ratio' ], ascending=False, axis=0)

,title,selftext,upvote_ratio,ups,downs,score,url
15,Slow traffic. He ran to the corner store for b...,Slow traffic. He ran to the corner store for b...,0.95,2912.0,0.0,2912.0,https://v.redd.it/shh9edqzpbg71
13,dick falls out while running,dick falls out while running,0.92,882.0,0.0,882.0,https://v.redd.it/6m0v1c4wubg71
21,Indian dudes chilling in car - Shadup,Indian dudes chilling in car - Shadup,0.91,328.0,0.0,328.0,https://v.redd.it/sgwl3juyvag71
3,What are men really thinking?,What are men really thinking?,0.97,145.0,0.0,145.0,https://v.redd.it/mby7qqjxdcg71
4,"Oh, so you can just walk up.","Oh, so you can just walk up.",0.95,139.0,0.0,139.0,https://v.redd.it/bs62f4e9dcg71
19,Okay this has a smidgen of NSFW.,Okay this has a smidgen of NSFW.,0.69,112.0,0.0,112.0,https://v.redd.it/ihnn3miogbg71
8,Guy from Boston yelling at a demon cat.,Guy from Boston yelling at a demon cat.,0.87,69.0,0.0,69.0,https://v.redd.it/mwlyhz1jbcg71
9,Street Fighter: Cats Edition.,Street Fighter: Cats Edition.,0.90,63.0,0.0,63.0,https://v.redd.it/ss3hesa59cg71
6,Goodluck to all students..,Goodluck to all students..,0.92,49.0,0.0,49.0,https://v.redd.it/n2s3b67tacg71
7,Bird be lagging.,Bird be lagging.,0.88,26.0,0.0,26.0,https://v.redd.it/nbx2pcttbcg71


# Trying out with top posts of today

In [28]:
reddit_api_url_top = 'https://oauth.reddit.com/r/funny/top'

In [29]:
res = requests.get(reddit_api_url_top, headers=headers, params={'limit':'100'})

In [30]:
res.json()

{'kind': 'Listing',
 'data': {'after': 't3_p0sfnb',
  'dist': 100,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'funny',
     'selftext': '',
     'author_fullname': 't2_8z3lr5f1',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': "I'm getting a bike",
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/funny',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 98,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_p0m6hl',
     'quarantine': False,
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.94,
     'author_flair_background_color': None,
     'subreddit_type': 'public',
     'ups': 30541,
     'total_awards_received': 16,
     'media_embed': {},
     'thumbnail_width': 140,
     'author_flair_template_id': None,
     'is_original_co

In [31]:
for post in res.json()['data']['children']:
    print(post['data']['title'])

I'm getting a bike
Guy shutting down on rollercoaster ride
Plot Twist: They're on Mars
Got married last week, our dog decided to steal the show by scooting his butt down the isle
Car maths
It’s a vibe.....
Seconds before disaster struck
Enovironmentalists are getting a bit more aggressive.
Car that parked next to me at
Rock em , Knock em , Sock em Featuring Jimmy smith the worst boxer
Now THAT's a brain freeze [OC]
Wanna know how kids talk? Here you go...
All these rules. I’m just trying to make a living.
Slow traffic. He ran to the corner store for beer.
"The reason they don't need a prequel is because they're already evil" [OC] 2021
Mission Impossible. Russian edition.
Was moving the sound system around and noticed it was rattling.The joys of having a toddler...
Fade me bro.
I turn 40 next weekend, I'll grow up then. This weekend though, I installed wifi to my garage door.
The kids got a new lego set.
Dude just rollin around with a Llama in the back. This is very South Dakota.
Indian

In [32]:
df_top = pd.DataFrame()

for post in res.json()['data']['children']:
    if post['data']['is_video']:
        df_top = df_top.append({
    #         'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['title'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'url': post['data']['url'],
        }, ignore_index=True)
    
df_top

,title,selftext,upvote_ratio,ups,downs,score,url
0,Guy shutting down on rollercoaster ride,Guy shutting down on rollercoaster ride,0.94,25450.0,0.0,25450.0,https://v.redd.it/botit5knj6g71
1,It’s a vibe.....,It’s a vibe.....,0.96,4997.0,0.0,4997.0,https://v.redd.it/hj8sxwbnl7g71
2,"Rock em , Knock em , Sock em Featuring Jimmy s...","Rock em , Knock em , Sock em Featuring Jimmy s...",0.96,1587.0,0.0,1587.0,https://v.redd.it/oujea73b99g71
3,Wanna know how kids talk? Here you go...,Wanna know how kids talk? Here you go...,0.95,950.0,0.0,950.0,https://v.redd.it/rnb7qrrm37g71
4,Slow traffic. He ran to the corner store for b...,Slow traffic. He ran to the corner store for b...,0.95,2941.0,0.0,2941.0,https://v.redd.it/shh9edqzpbg71
5,Mission Impossible. Russian edition.,Mission Impossible. Russian edition.,0.94,486.0,0.0,486.0,https://v.redd.it/vagqztvpj5g71
6,Fade me bro.,Fade me bro.,0.87,376.0,0.0,376.0,https://v.redd.it/616o8g8as8g71
7,"I turn 40 next weekend, I'll grow up then. Thi...","I turn 40 next weekend, I'll grow up then. Thi...",0.86,306.0,0.0,306.0,https://v.redd.it/7x9ej2g9s7g71
8,Indian dudes chilling in car - Shadup,Indian dudes chilling in car - Shadup,0.91,327.0,0.0,327.0,https://v.redd.it/sgwl3juyvag71
9,dick falls out while running,dick falls out while running,0.92,881.0,0.0,881.0,https://v.redd.it/6m0v1c4wubg71


In [33]:
df_top = df_top.sort_values(['score', 'upvote_ratio'], ascending=False, axis=0)
df_top

,title,selftext,upvote_ratio,ups,downs,score,url
0,Guy shutting down on rollercoaster ride,Guy shutting down on rollercoaster ride,0.94,25450.0,0.0,25450.0,https://v.redd.it/botit5knj6g71
1,It’s a vibe.....,It’s a vibe.....,0.96,4997.0,0.0,4997.0,https://v.redd.it/hj8sxwbnl7g71
4,Slow traffic. He ran to the corner store for b...,Slow traffic. He ran to the corner store for b...,0.95,2941.0,0.0,2941.0,https://v.redd.it/shh9edqzpbg71
2,"Rock em , Knock em , Sock em Featuring Jimmy s...","Rock em , Knock em , Sock em Featuring Jimmy s...",0.96,1587.0,0.0,1587.0,https://v.redd.it/oujea73b99g71
3,Wanna know how kids talk? Here you go...,Wanna know how kids talk? Here you go...,0.95,950.0,0.0,950.0,https://v.redd.it/rnb7qrrm37g71
9,dick falls out while running,dick falls out while running,0.92,881.0,0.0,881.0,https://v.redd.it/6m0v1c4wubg71
5,Mission Impossible. Russian edition.,Mission Impossible. Russian edition.,0.94,486.0,0.0,486.0,https://v.redd.it/vagqztvpj5g71
6,Fade me bro.,Fade me bro.,0.87,376.0,0.0,376.0,https://v.redd.it/616o8g8as8g71
8,Indian dudes chilling in car - Shadup,Indian dudes chilling in car - Shadup,0.91,327.0,0.0,327.0,https://v.redd.it/sgwl3juyvag71
7,"I turn 40 next weekend, I'll grow up then. Thi...","I turn 40 next weekend, I'll grow up then. Thi...",0.86,306.0,0.0,306.0,https://v.redd.it/7x9ej2g9s7g71


In [36]:
urls = list(df_top['url'])

In [37]:
urls

['https://v.redd.it/botit5knj6g71',
 'https://v.redd.it/hj8sxwbnl7g71',
 'https://v.redd.it/shh9edqzpbg71',
 'https://v.redd.it/oujea73b99g71',
 'https://v.redd.it/rnb7qrrm37g71',
 'https://v.redd.it/6m0v1c4wubg71',
 'https://v.redd.it/vagqztvpj5g71',
 'https://v.redd.it/616o8g8as8g71',
 'https://v.redd.it/sgwl3juyvag71',
 'https://v.redd.it/7x9ej2g9s7g71',
 'https://v.redd.it/wgzly3asc7g71',
 'https://v.redd.it/ihnn3miogbg71',
 'https://v.redd.it/kkan91l5m8g71',
 'https://v.redd.it/h2rksw2528g71',
 'https://v.redd.it/so99nb1me8g71',
 'https://v.redd.it/tva4zk4x47g71',
 'https://v.redd.it/1hkp2nyuh6g71',
 'https://v.redd.it/hxtoh6gag6g71',
 'https://v.redd.it/eottsmcor7g71',
 'https://v.redd.it/oztk5ky0b6g71',
 'https://v.redd.it/jzov8we6i8g71',
 'https://v.redd.it/183x66eg86g71',
 'https://v.redd.it/1l4irijub8g71']

In [38]:
from datetime import datetime


In [46]:
str(datetime.today().date())


'2021-08-09'

In [50]:
key={'PK': {'S': '2021-08-09'}, 'SK': {'S': 'funny'}}
resp = ddb.get_item(TableName='DailyUploadsTable-dev', Key=key)

In [51]:
resp

{'Item': {'SK': {'S': 'funny'},
  'PK': {'S': '2021-08-09'},
  'hello': {'S': 'world'}},
 'ResponseMetadata': {'RequestId': 'D5EBB0FN3UESQP7DU6FDTLLAIBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 09 Aug 2021 15:40:40 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '75',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'D5EBB0FN3UESQP7DU6FDTLLAIBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2040360102'},
  'RetryAttempts': 0}}

In [58]:
item={'PK': {'S': '2021-08-12'},
      'SK': {'S': 'videos'}, 
      'urls': {'L': [
          {'S': 'https:www.google.com'},
          {'S': 'https:www.facebook.com'},
          {'S': 'https:www.youtube.com'},
      ]}}
resp = ddb.put_item(TableName='DailyUploadsTable-dev', Item=item)

In [59]:
resp

{'ResponseMetadata': {'RequestId': 'TDDV33PCPI2J54LO17GSDIK1DBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 09 Aug 2021 15:52:00 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'TDDV33PCPI2J54LO17GSDIK1DBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}